In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from src.data import make_dataset
from src.features import build_features

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

HOSPITAL_A_ANLIZAR = 112103  # Este es el Torax


## Obtencion de Métricas de Egresos Hospitalarios

En este apartado se quieren calculcar diversas métricas relevantes para el análisis de los
egresos hospitalarios en Chile. Las métricas a calcular son las siguientes:

1. Cantidad de Egresos Hospitalarios
2. Dias de Estada Totales
3. Cantidad de Intervenciones Quirúrgicas
4. Cantidad de Muertos

Todas estas métricas serán calculadas para cada Año, Establecimiento de Salud y Diagnóstico
distinto que exista en la base de datos. Por ejemplo:

"En el año 2019, el Hospital San José (código XXXXXX) tuvo XX egresos, con XX dias estada promedio,
con XX intervenciones quirúrgicas y XX muertos para el diagnóstico A24.4"s

In [3]:
df_nacional = pl.scan_csv(
    "../data/processed/egresos_procesados.csv",
    infer_schema_length=10000,
    null_values="Extranjero",
)

In [4]:
AGRUPACION = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

metricas = build_features.obtener_metricas_egresos(
    df_nacional, AGRUPACION).collect()


In [5]:
display(metricas.sample(10))


ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,DIAG1,n_egresos,dias_estada_totales,n_int_q,n_muertos
i64,i64,str,str,u32,i64,i64,i64
2002,129100,"""Hospital Dr. M…","""E109""",6,43,0,0
2015,113130,"""Hospital Dr. E…","""C443""",4,4,0,0
2018,124120,"""Hospital de Fr…","""I951""",1,4,0,0
2004,110120,"""Hospital Dr. F…","""J157""",1,4,0,0
2010,115106,"""Hospital de Pi…","""B378""",1,5,0,0
2014,107108,"""Hospital Adria…","""O981""",1,2,0,0
2013,115107,"""Hospital San J…","""J188""",7,83,0,1
2001,116109,"""Hospital Dr. A…","""S219""",1,1,0,0
2019,117102,"""Hospital de Sa…","""K561""",1,5,1,0


- Al ver una muestra del calculo realizado, se puede observar la cantidad de egresos, dias de
estada, numero de intervenciones quirurgicas y cantidad de muertos para 10 duplas de
hospitales - diagnosticos distintos.

Si bien se tienen las metricas calculadas, ahora se quiere responder a las siguientes preguntas:

- En qué diagnósticos se destaca cada hospital productivamente? Esto significa, en qué diagnósticos
cada hospital tiene una mayor cantidad de egresos que otros hospitales?

- El hospital tiene un desempeño similar dentro de distintos grupos/estratos de hospitales?
El hospital se desempeñara de la misma forma nacionalmente, dentro del estrato público, privado,
etc...?

Para responder a la pregunta se realizará un ranking de todos los hospitales para cada
diagnóstico y cada año observado. Este ranking se realizará para distintos grupos/estratos de
hospitales (Nacional, Hospitales Públicos, Hospitales Privados, Hospitales GRD)

## Análisis de Estratos

In [7]:
dict_estratos = build_features.obtener_diccionario_estratos(
    df_nacional, HOSPITAL_A_ANLIZAR
)

In [8]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]

rankings_nacionales = build_features.agregar_ranking_estratos(
    metricas, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

rankings_hospital_analizado = rankings_nacionales.filter(
    pl.col("ESTABLECIMIENTO_SALUD") == HOSPITAL_A_ANLIZAR
)


Para hacer un contorl de lo que se calculo, se observará una muestra de 10 hospitales - diagnósticos
distintos

In [9]:
display(rankings_nacionales.sample(10))


ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,Capítulo,Sección,Categoría,Descripción,DIAG1,n_egresos,dias_estada_totales,n_int_q,n_muertos,ranking_nacionales_n_egresos,total_nacionales_n_egresos,%_nacionales_n_egresos,ranking_publicos_n_egresos,total_publicos_n_egresos,%_publicos_n_egresos,ranking_privados_n_egresos,total_privados_n_egresos,%_privados_n_egresos,ranking_grd_n_egresos,total_grd_n_egresos,%_grd_n_egresos,ranking_interno_n_egresos,total_interno_n_egresos,%_interno_n_egresos
i64,i64,str,str,str,str,str,str,u32,i64,i64,i64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64
2001,121113,"""Hospital Dr. E…","""Cap.10 ENFERM…","""J09-J18 GRIPE…","""J12 NEUMONÍA V…","""Neumonía debid…","""J128""",2,3,1,0,14,94,0.021277,11,81,0.024691,null,null,null,null,null,null,null,null,null
2018,108105,"""Hospital Psiqu…","""Cap.05 TRASTO…","""F40-F48 TRAST…","""F41 OTROS TRAS…","""Trastorno mixt…","""F412""",14,113,0,0,2,291,0.04811,1,151,0.092715,null,null,null,null,null,null,null,null,null
2015,112530,"""Hospital Milit…","""Cap.08 ENFERM…","""H80-H83 ENFER…","""H81 TRASTORNOS…","""Otros trastorn…","""H818""",1,4,0,0,15,26,0.038462,null,null,null,8,13,0.076923,null,null,null,null,null,null
2009,116112,"""Hospital Dr. B…","""Cap.18 SÍNTOM…","""R30-R39 SÍNTO…","""R31 HEMATURIA,…","""Hematuria, no …","""R31X""",4,11,0,0,113,1373,0.002913,86,1017,0.003933,null,null,null,null,null,null,null,null,null
2008,109201,"""Clínica Dávila…","""Cap.04 ENFERM…","""E00-E07 TRAST…","""E06 TIROIDITIS…","""Tiroiditis, no…","""E069""",8,13,3,0,1,43,0.186047,null,null,null,1,18,0.444444,null,null,null,null,null,null
2008,110120,"""Hospital Dr. F…","""Cap.12 ENFERM…","""L80-L99 OTROS…","""L81 OTROS TRAS…","""Hiperpigmentac…","""L810""",1,2,0,0,3,3,0.333333,2,2,0.5,null,null,null,1,1,1.0,null,null,null
2010,103220,"""Clínica La Por…","""Cap.15 EMBARA…","""O85-O92 COMPL…","""O90 COMPLICACI…","""Dehiscencia de…","""O901""",1,1,1,0,64,285,0.003509,null,null,null,9,11,0.090909,null,null,null,null,null,null
2017,113100,"""Hospital Barro…","""Cap.18 SÍNTOM…","""R00-R09 SÍNTO…","""R04 HEMORRAGIA…","""Epistaxis""","""R040""",8,34,1,0,18,670,0.01194,10,444,0.018018,null,null,null,9,276,0.028986,null,null,null
2008,126100,"""Hospital Clíni…","""Cap.01 CIERTA…","""A00-A09 ENFER…","""A08 INFECCIONE…","""Enteritis debi…","""A080""",2,11,0,0,88,1756,0.001139,54,960,0.002083,null,null,null,38,834,0.002398,null,null,null


En la muestra anterior se puede observar las métricas previamente calculada, y el ranking
respectivo que tiene cada hospital para diagnósticos en cada año específico.

In [10]:
rankings_nacionales.to_pandas().to_csv(
    "../data/interim/ranking_nacional_egresos.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)

rankings_hospital_analizado.to_pandas().to_csv(
    "../data/interim/ranking_torax_egresos.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)

## Metricas para personas sobre 14 años

En este análisis se quiere ver el ranking de egresos pero solamente tomando en cuenta a personas
mayores a 14 años. Esto, para dilucidar si el hospital del tórax es altamente relevante para tratar
patologías congénitas de adultos.

In [11]:
df_nacional_mayores_a_14 = df_nacional.filter(pl.col("EDAD_A_OS") > 14)

In [12]:
metricas_sobre_14 = build_features.obtener_metricas_egresos(
    df_nacional_mayores_a_14, AGRUPACION
).collect()


In [13]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]

rankings_nacionales_sobre_14 = build_features.agregar_ranking_estratos(
    metricas_sobre_14, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

diags_congenitos_relevantes = rankings_nacionales_sobre_14.filter(
    (pl.col("DIAG1").is_in(["Q230", "Q211", "Q231"])) & (
        pl.col("ANO_EGRESO") == 2019)
)


In [14]:
diags_congenitos_relevantes.to_pandas().to_csv(
    "../data/interim/diags_congenitas_sobre_14_egresos.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)